# SQD postprocessing for ATP

In [74]:
fragment = "atp_0_be2_f4"

all_adapt_iterations = [1, 2]

## Setup

In [75]:
import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})
import numpy as np
import pickle
import glob

import pyscf.tools
from pyscf import ao2mo

import collections
from functools import partial
import os
import pickle

from qiskit.primitives import BitArray
from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

In [76]:
def transform_bitstring(bits):
    """
    Convert a given bitstring from Openfermion convention 
    (alternating alpha/beta, big endian) to Qiskit (all alpha
    then all beta, little endian).
    """

    left = [bits[i] for i in range(len(bits)) if i % 2 == 1]   # beta
    right = [bits[i] for i in range(len(bits)) if i % 2 == 0]  # alpha

    # Reverse each half
    left.reverse()
    right.reverse()

    # Concatenate
    return ''.join(left + right)

In [77]:
# import qiskit.visualization


# qiskit.visualization.plot_histogram(
#     all_counts_hardware[0],
#     # target_string=hartree_fock_bitstring,
#     # sort="hamming",
#     number_to_keep=10,
#     figsize=(7, 8),
#     title=computer.name,
# )

In [78]:
circuit_dir = "circuits"
hamiltonian_dir = "hamiltonians"

In [79]:
fcidump = pyscf.tools.fcidump.read(f"{hamiltonian_dir}/{fragment}.fcidump")

Parsing hamiltonians/atp_0_be2_f4.fcidump


In [80]:
n_orbitals = fcidump.get("NORB")
num_electrons = fcidump.get("NELEC")
ecore = fcidump.get("ECORE")
h1 = fcidump.get("H1")
h2 = fcidump.get("H2")
h2 = ao2mo.restore(1, h2, n_orbitals)

In [81]:
nqubits = 2 * n_orbitals

In [82]:
results_dir = "results"

In [83]:
all_counts = []
fnames = []

for adapt_iterations in all_adapt_iterations:
    fname = glob.glob(f"{results_dir}/{fragment}/*{adapt_iterations:03d}*.qasm*")[0]
    counts = pickle.load(
        open(f"{fname}", "rb")
    )
    mode_order = pickle.load(
        open(f"{circuit_dir}/{fragment}/{fragment}_mode_order_{adapt_iterations:03d}_adaptiterations.pkl", "rb")
    )
    qubit_order = pickle.load(
        open(f"{circuit_dir}/{fragment}/{fragment}_qubit_order_{adapt_iterations:03d}_adaptiterations.pkl", "rb")
    )

    measurement_outcomes = counts
    permuted_outcomes = {}
    for original_bitstring in measurement_outcomes.keys():
        qubit_permuted_bitstring = "".join([original_bitstring[qubit_order.index(n)] for n in range(nqubits)])
        mode_permuted_bitstring = "".join([qubit_permuted_bitstring[mode_order.index(n)] for n in range(nqubits)])

        final_permuted_bitstring = transform_bitstring(mode_permuted_bitstring)
        permuted_outcomes[final_permuted_bitstring[::]] = measurement_outcomes[original_bitstring]
    
    counts = permuted_outcomes
    all_counts.append(counts)

    print("ADAPT iteration", adapt_iterations)
    max_key = max(counts, key=counts.get)
    print(f'Most common bitstring: {max_key} with count {counts[max_key]}')
    print(f'Total number of bitstrings: {len(counts)}')
    print(f"Total number of samples:", sum(counts.values()))

ADAPT iteration 1
Most common bitstring: 0000000000000000111111111111111100000000000000001111111111111111 with count 41125
Total number of bitstrings: 1926
Total number of samples: 100000
ADAPT iteration 2
Most common bitstring: 0000000000000000111111111111111100000000000000001111111111111111 with count 37918
Total number of bitstrings: 2613
Total number of samples: 100000


In [84]:
# TODO: Implement strategies to cap the number of shots when concatenating.
counts = collections.Counter()
for c in all_counts:
    for bitstring, count in c.items():
        counts[bitstring] += count

In [85]:
sum(counts.values())

200000

In [86]:
bit_array = BitArray.from_counts(counts)

In [87]:
energy_tol = 1e-8
occupancies_tol = 1e-8
carryover_threshold = 1e-5

sci_solver = partial(solve_sci_batch, spin_sq=0, max_cycle=10000)
result_history = []

def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + ecore}")
        print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")


result = diagonalize_fermionic_hamiltonian(
    one_body_tensor=h1,
    two_body_tensor=h2,
    bit_array=bit_array,
    samples_per_batch=500,
    norb=n_orbitals,
    nelec=(num_electrons // 2, num_electrons // 2),
    num_batches=2,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=100,
    sci_solver=sci_solver,
    symmetrize_spin=True,
    carryover_threshold=carryover_threshold,
    callback=callback,
)

Iteration 1
	Subsample 0
		Energy: -261.8116206022048
		Subspace dimension: 38416
	Subsample 1
		Energy: -261.8116206022048
		Subspace dimension: 38416
Iteration 2
	Subsample 0
		Energy: -261.8468641903113
		Subspace dimension: 69169
	Subsample 1
		Energy: -261.8468641903113
		Subspace dimension: 69169
Iteration 3
	Subsample 0
		Energy: -261.85616403303584
		Subspace dimension: 74529
	Subsample 1
		Energy: -261.85616403303584
		Subspace dimension: 74529
Iteration 4
	Subsample 0
		Energy: -261.86130074139237
		Subspace dimension: 75625
	Subsample 1
		Energy: -261.86130074139237
		Subspace dimension: 75625
Iteration 5
	Subsample 0
		Energy: -261.8613067564842
		Subspace dimension: 76729
	Subsample 1
		Energy: -261.8613067564842
		Subspace dimension: 76729
Iteration 6
	Subsample 0
		Energy: -261.86132526803834
		Subspace dimension: 78400
	Subsample 1
		Energy: -261.86132526803834
		Subspace dimension: 78400
Iteration 7
	Subsample 0
		Energy: -261.8613304933385
		Subspace dimension: 80089


In [88]:
min_e = [
    min(result, key=lambda res: res.energy).energy + ecore
    for result in result_history
]
min_e

[np.float64(-261.8116206022048),
 np.float64(-261.8468641903113),
 np.float64(-261.85616403303584),
 np.float64(-261.86130074139237),
 np.float64(-261.8613067564842),
 np.float64(-261.86132526803834),
 np.float64(-261.8613304933385),
 np.float64(-261.8613304933385)]

In [89]:
iterations_key = "_".join(map(str, all_adapt_iterations))
save_name = f"sqd_energies_{iterations_key}_adaptiterations.txt"
save_name

'sqd_energies_1_2_adaptiterations.txt'

In [90]:
np.savetxt(save_name, min_e)